In [1]:
import yolov5
import cv2
from PIL import Image
from io import BytesIO
from transformers import pipeline
import torch

/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load video data
cap = cv2.VideoCapture("dashcamtest.mov")

# Load YOLOv5 model
model = yolov5.load("yolov5s.pt")

# Load LLM model
pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    device="mps",  # replace with "mps" to run on a Mac device
)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.61s/it]


In [3]:
messages = [
    {"role": "user", "content": "who are you?"},
]

pipe(messages)

[{'generated_text': [{'role': 'user', 'content': 'who are you?'},
   {'role': 'assistant', 'content': 'I am Gemma, an open-'}]}]

In [ ]:
# Define grid dimensions (e.g., 3x3 grid)
grid_rows = 5
grid_cols = 5

# Define class labels (example for COCO dataset)
class_labels = model.names

# Define the class IDs for vehicles (e.g., car, motorcycle, bus, truck)
vehicle_class_ids = [2, 3, 5, 7]  # Car, Motorcycle, Bus, Truck

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Cannot read from video.")
        break

    # Get frame dimensions
    frame_height, frame_width = frame.shape[:2]

    # Calculate grid cell size
    cell_width = frame_width // grid_cols
    cell_height = frame_height // grid_rows

    # Draw the grid on the frame
    for i in range(1, grid_cols):
        cv2.line(frame, (i * cell_width, 0), (i * cell_width, frame_height), (0, 255, 0), 2)
    for i in range(1, grid_rows):
        cv2.line(frame, (0, i * cell_height), (frame_width, i * cell_height), (0, 255, 0), 2)

    # Convert the frame to a PIL image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)

    # Perform object detection on the frame
    results = model(pil_image)

    # Draw the bounding boxes and check grid cells
    for result in results.xyxy[0]:
        x1, y1, x2, y2, conf, class_id = result
        class_id = int(class_id)
        if class_id in vehicle_class_ids:
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # Add label
            label = f"{class_labels[class_id]}: {conf:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Display the frame with grid and bounding boxes
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("Exiting...")
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIow

Exiting...


/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


: 